In [18]:
import pandas as pd

df = pd.read_csv('Financial_inclusion_dataset.csv')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23524 entries, 0 to 23523
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   country                 23524 non-null  object
 1   year                    23524 non-null  int64 
 2   uniqueid                23524 non-null  object
 3   bank_account            23524 non-null  object
 4   location_type           23524 non-null  object
 5   cellphone_access        23524 non-null  object
 6   household_size          23524 non-null  int64 
 7   age_of_respondent       23524 non-null  int64 
 8   gender_of_respondent    23524 non-null  object
 9   relationship_with_head  23524 non-null  object
 10  marital_status          23524 non-null  object
 11  education_level         23524 non-null  object
 12  job_type                23524 non-null  object
dtypes: int64(3), object(10)
memory usage: 2.3+ MB
None


In [19]:
print(df.isnull().sum())

country                   0
year                      0
uniqueid                  0
bank_account              0
location_type             0
cellphone_access          0
household_size            0
age_of_respondent         0
gender_of_respondent      0
relationship_with_head    0
marital_status            0
education_level           0
job_type                  0
dtype: int64


In [20]:
df.drop_duplicates(inplace=True)


In [21]:
def detect_outliers_iqr(column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    outliers = df[(df[column] < lower) | (df[column] > upper)]
    print(f"\nOutliers in {column}: {len(outliers)}")
    return outliers

# Check for outliers
detect_outliers_iqr('household_size')
detect_outliers_iqr('age_of_respondent')



Outliers in household_size: 381

Outliers in age_of_respondent: 241


,country,year,uniqueid,bank_account,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
209,Kenya,2018,uniqueid_210,No,Rural,No,3,95,Female,Parent,Widowed,No formal education,Informally employed
268,Kenya,2018,uniqueid_269,No,Urban,Yes,1,85,Female,Head of Household,Widowed,Primary education,Remittance Dependent
340,Kenya,2018,uniqueid_341,No,Rural,No,2,95,Male,Head of Household,Married/Living together,Primary education,Farming and Fishing
366,Kenya,2018,uniqueid_367,No,Rural,No,1,86,Female,Head of Household,Widowed,Primary education,Remittance Dependent
370,Kenya,2018,uniqueid_371,No,Rural,Yes,3,95,Male,Head of Household,Married/Living together,Primary education,Other Income
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22995,Uganda,2018,uniqueid_1573,No,Rural,No,4,95,Female,Spouse,Married/Living together,Primary education,No Income
23053,Uganda,2018,uniqueid_1631,No,Urban,No,2,98,Female,Head of Household,Widowed,No formal education,No Income
23352,Uganda,2018,uniqueid_1930,No,Rural,Yes,3,86,Male,Head of Household,Divorced/Seperated,No formal education,No Income
23395,Uganda,2018,uniqueid_1976,No,Rural,No,2,85,Female,Head of Household,Divorced/Seperated,No formal education,Other Income


In [22]:
#pip install scikit-learn


In [23]:

from sklearn.preprocessing import LabelEncoder

# List of categorical columns to encode
categorical_columns = ['country', 'gender_of_respondent', 'education_level', 'job_type', 'location_type', 'cellphone_access', 'relationship_with_head', 'marital_status']

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Encode categorical columns
for col in categorical_columns:
    df[col] = label_encoder.fit_transform(df[col])

In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Define features (X) and target (y)
X = df.drop(columns=['bank_account', 'uniqueid'])  # Drop the target column
y = df['bank_account']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize the RandomForest model
model = RandomForestClassifier(random_state=42, max_depth=5, min_samples_split=10)

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Optionally, evaluate model
from sklearn.metrics import accuracy_score
print("Accuracy: ", accuracy_score(y_test, y_pred))


Accuracy:  0.8822612638141116


In [ ]:
import pickle
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)

